# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [63]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [64]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/danieldacosta/Documents/GitHub/nosql-apache-cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [65]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [66]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [67]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [69]:
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkifyks
        WITH REPLICATION = 
        {'class': 'SimpleStrategy', 'replication_factor': 1}
    """)

except Exception as e:
    print(e)

#### Set Keyspace

In [70]:
session.set_keyspace('sparkifyks')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [71]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS history_by_session_and_item(
            history_id timeuuid,
            session_id int,
            item_in_session int,
            artist text,
            song text,
            length decimal,
            PRIMARY KEY ((history_id, session_id, item_in_session))
        );
    """)

except Exception as e:
    print(e)

In [72]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO history_by_session_and_item(history_id, session_id, item_in_session, artist, song, length)"
        query = query + "values(now(), %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [73]:
result = session.execute("""
    SELECT artist, song, length
    FROM history_by_session_and_item
    WHERE
        session_id=338 AND
        item_in_session=4
        AND history_id = 0a47ffa0-0779-11eb-96e8-6d2c86545d91;
""")

for row in result:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [74]:
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS song_info_by_user_and_session(
            session_id int,
            user_id int,
            item_in_session int,
            artist text,
            song text,
            first_name text,
            last_name text,
            PRIMARY KEY ((session_id, user_id), item_in_session)
        );
    """)

except Exception as e:
    print(e)

In [75]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO song_info_by_user_and_session(session_id, user_id, item_in_session, artist, song, first_name, last_name)"
        query = query + "values(%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[10]), int(line[3]), line[0], line[9], line[1], line[4]))

In [76]:
result = session.execute("""
    SELECT artist, song, first_name, last_name
    FROM song_info_by_user_and_session
    WHERE
        session_id=182 AND
        user_id=10
    ORDER BY item_in_session;
""")

for row in result:
    print(row.artist, row.song, row.first_name, row.last_name)

Down To The Bone Keep On Keepin&#39; On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron &amp; Max Sanna Radio Edit) Sylvie Cruz


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

                    

In [77]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS user_info_by_song(
            user_info_id timeuuid,
            song text,
            first_name text,
            last_name text,
            PRIMARY KEY (user_info_id, song)
        );
    """)

except Exception as e:
    print(e)

In [78]:

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_info_by_song(user_info_id, song, first_name, last_name)"
        query = query + "values(now(), %s, %s, %s)"
        session.execute(query, (line[9], line[1], line[4]))

In [79]:
result = session.execute("""
    SELECT first_name, last_name
    FROM user_info_by_song
    WHERE
        song='All Hands Against His Own'
        AND user_info_id in (065c49b0-077d-11eb-96e8-6d2c86545d91, 0514fca0-077d-11eb-96e8-6d2c86545d91, 02fb1d50-077d-11eb-96e8-6d2c86545d91);
""")

for row in result:
    print(row.first_name, row.last_name)

Tegan Levine
Sara Johnson
Jacqueline Lynch


### Drop the tables before closing out the sessions

In [80]:
try:
    session.execute("DROP TABLE history_by_session_and_item")
    session.execute("DROP TABLE song_info_by_user_and_session")
    session.execute("DROP TABLE user_info_by_song")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [81]:
session.shutdown()
cluster.shutdown()